In [1]:
!pip install PyPDF2 deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00


In [2]:
import zipfile
import os

zip_path = "/content/CONTRACTS.zip"
extract_path = "/content/contracts_extracted"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

extracted_files = [f for f in os.listdir(extract_path) if f.lower().endswith(".pdf")]
extracted_files

['COR3.pdf', 'COR2.pdf', 'PER3.pdf', 'PER2.pdf', 'COR1.pdf', 'PER1.pdf']

In [3]:
from PyPDF2 import PdfReader
import pandas as pd
import re
import os
from deep_translator import GoogleTranslator

contract_paths = {f: os.path.join(extract_path, f) for f in extracted_files}
corporate_paths = {k: v for k, v in contract_paths.items() if k.startswith("COR")}
personal_paths = {k: v for k, v in contract_paths.items() if k.startswith("PER")}

def translate_purpose(text):
    if text == "Not Found" or not text.strip():
        return text
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception as e:
        print(f"Translation failed for: {text} | Error: {e}")
        return text

def extract_between(text, before, after):
    try:
        return text.split(before, 1)[1].split(after, 1)[0].strip()
    except:
        return "Not Found"

def extract_regex(text, pattern, multiple=False):
    matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
    return ", ".join(matches) if multiple and matches else matches[0].strip() if matches else "Not Found"

def correct_percentage(val):
    try:
        if val == "Not Found": return val
        num = float(val.replace("%", "").replace(",", ""))
        if num > 100: num /= 100
        return f"{num:.2f}%"
    except: return "Not Found"

def extract_amount_with_currency(text, pattern):
    m = re.search(pattern + r"\s*([\d.,]+)\s*\(\s*.*?\)\s*([A-Z]{2,4})", text, re.I | re.DOTALL)
    return f"{m.group(1)} {m.group(2)}" if m else "Not Found"

patterns = {
    "Lender Company": ("1. KREDIDHËNËSI:", ", Institucion Financiar Jobankë"),
    "Lender CEO Name": ("Tiranë me Administrator", ", i përfaqësuar në këtë kontratë me autorizim nga"),
    "Loan Term": ("kredisë është për një periudhë prej", "muaj"),
    "Purpose of Credit": ("2.4 Kjo kredi jepet për", "2.5"),
}

def extract_shared(text):
    return {
        "Borrower Name": extract_regex(text, r"KREDIMARRËSI\s+Z/Znj\.?\s+(.*?)\s+i/e\s+biri/bija"),
        "Loan Amount": extract_amount_with_currency(text, r"shumën:"),
        "Application Fees": extract_amount_with_currency(text, r"komisionit.*?shumën"),
        "Rate of Interest": correct_percentage(extract_regex(text, r"interes vjetor.*?([\d.,]+)")),
        "NEI Rate": correct_percentage(extract_regex(text, r"NEI.*?masën.*?([\d.,]+)")),
        "Penalty": extract_regex(text, r"barabartë me\s*([\d.,]+%)\s+në\s+ditë")
    }

def extract_corporate(text):
    d = extract_shared(text)
    d.update({
        "Corporate who Borrows": extract_regex(text, r"KREDIMARRËSI\s+(.*?)\s*,\s+i\s+them"),
        "Director of Borrowing Company": extract_regex(text, r"nga\s+Z\.\s+([A-ZÇËË]+\s+[A-ZÇËË]+\s+[A-ZÇËË]+)"),
        "Guarantors": extract_regex(text, r"DORËZANËS/HIPOTEKUES\s+Z/Znj\.\s+([A-ZÇËË]+\s+[A-ZÇËË]+\s+[A-ZÇËË]+)", multiple=True),
    })
    return d

def process_contracts(paths, patterns, extract_fn, columns):
    rows = []
    for name, path in paths.items():
        text = "\n".join(p.extract_text() for p in PdfReader(path).pages if p.extract_text())
        row = {"Contract": name}
        for field, (before, after) in patterns.items():
            val = extract_between(text, before, after)
            row[field] = translate_purpose(val) if field == "Purpose of Credit" else val
        row.update(extract_fn(text))
        for col in columns:
            row.setdefault(col, "Not Found")
        rows.append(row)
    return pd.DataFrame(rows)[["Contract"] + columns]

corp_cols = [
    "Lender Company", "Lender CEO Name", "Borrower Name", "Corporate who Borrows",
    "Director of Borrowing Company", "Guarantors", "Loan Amount", "Rate of Interest",
    "Application Fees", "Loan Term", "Purpose of Credit", "NEI Rate", "Penalty"
]

pers_cols = [
    "Lender Company", "Lender CEO Name", "Borrower Name", "Loan Amount",
    "Rate of Interest", "Application Fees", "Loan Term", "Purpose of Credit",
    "NEI Rate", "Penalty"
]

df_corp = process_contracts(corporate_paths, patterns, extract_corporate, corp_cols)
df_pers = process_contracts(personal_paths, patterns, extract_shared, pers_cols)

df_corp.sort_values(by="Contract", inplace=True)
df_pers.sort_values(by="Contract", inplace=True)

df_corp.to_excel("/content/Final_Corporate.xlsx", index=False)
df_pers.to_excel("/content/Final_Personal.xlsx", index=False)


In [4]:
df_corp

,Contract,Lender Company,Lender CEO Name,Borrower Name,Corporate who Borrows,Director of Borrowing Company,Guarantors,Loan Amount,Rate of Interest,Application Fees,Loan Term,Purpose of Credit,NEI Rate,Penalty
2,COR1.pdf,FONDI BESA SHA,Z.Bajram MUÇAJ,Not Found,Leze Lufaj PF PERSON FIZIK TREGTAR (PF),Leze Lulash Lufaj,"Nikë Lulash Lufaj, Leze Lulash Lufaj","2,350,000.00 LEK",20.40%,"70,500.00 LEK",60,investment,24.20%,0.35%
1,COR2.pdf,FONDI BESA SHA,Z.Bajram MUÇAJ,Belma Konstruksion Shpk,Not Found,Not Found,Not Found,"15,000.00 EUR",18.50%,150.00 EUR,12,Circulatory capital,22.30%,0.35%
0,COR3.pdf,FONDI BESA SHA,Z.Bajram MUÇAJ,QENDRA E NDERTIMIT,Not Found,Not Found,Not Found,"48,000.00 EUR",15.60%,"1,920.00 EUR",48,Migration,19.40%,0.25%


In [5]:
df_pers

,Contract,Lender Company,Lender CEO Name,Borrower Name,Loan Amount,Rate of Interest,Application Fees,Loan Term,Purpose of Credit,NEI Rate,Penalty
2,PER1.pdf,FONDI BESA SHA,Z.Bajram MUÇAJ,Xhonatan Matei PF,"5,000,000.00 LEK",18.00%,"75,000.00 LEK",12,investment,23.05%,0.25%
1,PER2.pdf,FONDI BESA SHA,Z.Bajram MUÇAJ,Vasilika Dhimitraq Kinolli,"50,000.00 LEK",29.00%,"1,500.00 LEK",12,Personnel,41.29%,0.85%
0,PER3.pdf,FONDI BESA SHA,Z.Bajram MUÇAJ,Fitim Maliq Dervishi,"200,000.00 LEK",15.30%,"4,000.00 LEK",18,"Bathroom renovation, repair of yard, garage or...",20.30%,0.85%
